## Distributed Deep Learning on CPU-GPU

### What is Deep Learning?
Deep learning is a machine learning technique that teaches computers to do what comes naturally to humans: learn by example. Deep learning is a key technology behind driverless cars, enabling them to recognize a stop sign, or to distinguish a pedestrian from a lamppost. It is the key to voice control in consumer devices like phones, tablets, TVs, and hands-free speakers. Deep learning is getting lots of attention lately and for good reason. It’s achieving results that were not possible before.

In deep learning, a computer model learns to perform classification tasks directly from images, text, or sound. Deep learning models can achieve state-of-the-art accuracy, sometimes exceeding human-level performance. Models are trained by using a large set of labeled data and neural network architectures that contain many layers.

### Awesome! But more in detail?
Deep Learning is a subfield of Artificial Intelligence and then of Machine learning concerned with algorithms inspired by the structure and function of the brain called artificial neural networks.
The *Deep* stays for *Deep Neural Network*. 
In other words the Deep learning is a Machine learning area where neural networks are composed by *stacked layers of neurons of different types*

Due to Deep learning is inspired to the brain, Deep learning Neural Network have different type. 
The most common are:

+ Dense Neural Network (evolution of MLP classifier)
+ Convolutional Neural Network ( mainly used for vision/recognition purposes, they emulate the visual brain cortex)
+ Recurrent Neural Network (mainly used for timeseries forcasting, text / audio elaboration)

Deep Neural Network works fine with clustering because, due to their computation complexity, they are specifically designed for parallel computation (CPU/ GPU with CUDA) and distrbuted computing since they are implemented as graph.

Unfortunately today we will see only how to distribute the computation over a cluster with CPU since there aren't available GPUs on the lab.
Anyway, to work with CUDA on your PC, it's sufficient to install CUDA and the package tensorflow-gpu in a propper way.
Drop me an email if you'd like to install it at home: [stefano.campese@omnys.com](stefano.campese@omnys.com)



### How to work with Distributed Deep learning ?

In order to work in a simple way with clustering and deep lerning three packages are necessary:
+ Tensorflow
+ Keras
+ Dask

### What is Tensorflow?

TensorFlow is an open source software library for numerical computation using data-flow graphs. It was originally developed by the Google Brain Team within Google's Machine Intelligence research organization for machine learning and deep neural networks research, but the system is general enough to be applicable in a wide variety of other domains as wel.

### What is Keras?

Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

Use Keras if you need a deep learning library that:

Allows for easy and fast prototyping (through user friendliness, modularity, and extensibility).
Supports both convolutional networks and recurrent networks, as well as combinations of the two.
Runs seamlessly on CPU and GPU.

Ok, Let's see how to create simple Distributed Deep Dense Neural Network

In [9]:
from dask.distributed import Client
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from dask_ml.datasets import make_classification

client = Client('192.168.9.30:8786')

In [10]:
from dask_tensorflow import start_tensorflow
tf_spec, dask_spec = start_tensorflow(client, ps=2, worker=2)




In [11]:
from dask_ml.model_selection import train_test_split

X, y = make_classification(n_samples=6000, n_features=10, random_state=42, chunks=1000)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [12]:
def build_deep_neural_network():
    # create model
    
    model = Sequential()
    model.add(Dense(60, input_shape=(10,), kernel_initializer='normal', activation='relu')) #layer 1 with 60 neurons
    model.add(Dense(30, kernel_initializer='normal', activation='relu')) #layer 2 with 30 neurons
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid')) #layer 3 with 1 neuron (binary classification)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:
from sklearn.metrics import accuracy_score
model = build_deep_neural_network()

model.fit(X_train, y_train, verbose=1, epochs=1)
scores = model.evaluate(X_test,y_test, verbose=1)
print("Accuracy: "+str(scores[1]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1


InternalError: Blas GEMM launch failed : a.shape=(32, 10), b.shape=(10, 60), m=32, n=60, k=10
	 [[{{node dense_1/MatMul}}]]
	 [[{{node loss/mul}}]]

Let's try to look the cluster dashboard! Take a look to the Graph tab!

### Number recognition: let's due a magic!

In [14]:
import dask.dataframe as dd

df = dd.read_csv('https://www.dropbox.com/s/366wxe7458yixl4/numbers.csv?dl=1')

df

,label,Pixel 1,Pixel 2,Pixel 3,Pixel 4,Pixel 5,Pixel 6,Pixel 7,Pixel 8,Pixel 9,Pixel 10,Pixel 11,Pixel 12,Pixel 13,Pixel 14,Pixel 15,Pixel 16,Pixel 17,Pixel 18,Pixel 19,Pixel 20,Pixel 21,Pixel 22,Pixel 23,Pixel 24,Pixel 25,Pixel 26,Pixel 27,Pixel 28,Pixel 29,Pixel 30,Pixel 31,Pixel 32,Pixel 33,Pixel 34,Pixel 35,Pixel 36,Pixel 37,Pixel 38,Pixel 39,Pixel 40,Pixel 41,Pixel 42,Pixel 43,Pixel 44,Pixel 45,Pixel 46,Pixel 47,Pixel 48,Pixel 49,Pixel 50,Pixel 51,Pixel 52,Pixel 53,Pixel 54,Pixel 55,Pixel 56,Pixel 57,Pixel 58,Pixel 59,Pixel 60,Pixel 61,Pixel 62,Pixel 63,Pixel 64,Pixel 65,Pixel 66,Pixel 67,Pixel 68,Pixel 69,Pixel 70,Pixel 71,Pixel 72,Pixel 73,Pixel 74,Pixel 75,Pixel 76,Pixel 77,Pixel 78,Pixel 79,Pixel 80,Pixel 81,Pixel 82,Pixel 83,Pixel 84,Pixel 85,Pixel 86,Pixel 87,Pixel 88,Pixel 89,Pixel 90,Pixel 91,Pixel 92,Pixel 93,Pixel 94,Pixel 95,Pixel 96,Pixel 97,Pixel 98,Pixel 99,Pixel 100,Pixel 101,Pixel 102,Pixel 103,Pixel 104,Pixel 105,Pixel 106,Pixel 107,Pixel 108,Pixel 109,Pixel 110,Pixel 111,Pixel 112,Pixel 113,Pixel 114,Pixel 115,Pixel 116,Pixel 117,Pixel 118,Pixel 119,Pixel 120,Pixel 121,Pixel 122,Pixel 123,Pixel 124,Pixel 125,Pixel 126,Pixel 127,Pixel 128,Pixel 129,Pixel 130,Pixel 131,Pixel 132,Pixel 133,Pixel 134,Pixel 135,Pixel 136,Pixel 137,Pixel 138,Pixel 139,Pixel 140,Pixel 141,Pixel 142,Pixel 143,Pixel 144,Pixel 145,Pixel 146,Pixel 147,Pixel 148,Pixel 149,Pixel 150,Pixel 151,Pixel 152,Pixel 153,Pixel 154,Pixel 155,Pixel 156,Pixel 157,Pixel 158,Pixel 159,Pixel 160,Pixel 161,Pixel 162,Pixel 163,Pixel 164,Pixel 165,Pixel 166,Pixel 167,Pixel 168,Pixel 169,Pixel 170,Pixel 171,Pixel 172,Pixel 173,Pixel 174,Pixel 175,Pixel 176,Pixel 177,Pixel 178,Pixel 179,Pixel 180,Pixel 181,Pixel 182,Pixel 183,Pixel 184,Pixel 185,Pixel 186,Pixel 187,Pixel 188,Pixel 189,Pixel 190,Pixel 191,Pixel 192,Pixel 193,Pixel 194,Pixel 195,Pixel 196,Pixel 197,Pixel 198,Pixel 199,Pixel 200,Pixel 201,Pixel 202,Pixel 203,Pixel 204,Pixel 205,Pixel 206,Pixel 207,Pixel 208,Pixel 209,Pixel 210,Pixel 211,Pixel 212,Pixel 213,Pixel 214,Pixel 215,Pixel 216,Pixel 217,Pixel 218,Pixel 219,Pixel 220,Pixel 221,Pixel 222,Pixel 223,Pixel 224,Pixel 225,Pixel 226,Pixel 227,Pixel 228,Pixel 229,Pixel 230,Pixel 231,Pixel 232,Pixel 233,Pixel 234,Pixel 235,Pixel 236,Pixel 237,Pixel 238,Pixel 239,Pixel 240,Pixel 241,Pixel 242,Pixel 243,Pixel 244,Pixel 245,Pixel 246,Pixel 247,Pixel 248,Pixel 249,Pixel 250,Pixel 251,Pixel 252,Pixel 253,Pixel 254,Pixel 255,Pixel 256,Pixel 257,Pixel 258,Pixel 259,Pixel 260,Pixel 261,Pixel 262,Pixel 263,Pixel 264,Pixel 265,Pixel 266,Pixel 267,Pixel 268,Pixel 269,Pixel 270,Pixel 271,Pixel 272,Pixel 273,Pixel 274,Pixel 275,Pixel 276,Pixel 277,Pixel 278,Pixel 279,Pixel 280,Pixel 281,Pixel 282,Pixel 283,Pixel 284,Pixel 285,Pixel 286,Pixel 287,Pixel 288,Pixel 289,Pixel 290,Pixel 291,Pixel 292,Pixel 293,Pixel 294,Pixel 295,Pixel 296,Pixel 297,Pixel 298,Pixel 299,Pixel 300,Pixel 301,Pixel 302,Pixel 303,Pixel 304,Pixel 305,Pixel 306,Pixel 307,Pixel 308,Pixel 309,Pixel 310,Pixel 311,Pixel 312,Pixel 313,Pixel 314,Pixel 315,Pixel 316,Pixel 317,Pixel 318,Pixel 319,Pixel 320,Pixel 321,Pixel 322,Pixel 323,Pixel 324,Pixel 325,Pixel 326,Pixel 327,Pixel 328,Pixel 329,Pixel 330,Pixel 331,Pixel 332,Pixel 333,Pixel 334,Pixel 335,Pixel 336,Pixel 337,Pixel 338,Pixel 339,Pixel 340,Pixel 341,Pixel 342,Pixel 343,Pixel 344,Pixel 345,Pixel 346,Pixel 347,Pixel 348,Pixel 349,Pixel 350,Pixel 351,Pixel 352,Pixel 353,Pixel 354,Pixel 355,Pixel 356,Pixel 357,Pixel 358,Pixel 359,Pixel 360,Pixel 361,Pixel 362,Pixel 363,Pixel 364,Pixel 365,Pixel 366,Pixel 367,Pixel 368,Pixel 369,Pixel 370,Pixel 371,Pixel 372,Pixel 373,Pixel 374,Pixel 375,Pixel 376,Pixel 377,Pixel 378,Pixel 379,Pixel 380,Pixel 381,Pixel 382,Pixel 383,Pixel 384,Pixel 385,Pixel 386,Pixel 387,Pixel 388,Pixel 389,Pixel 390,Pixel 391,Pixel 392,Pixel 393,Pixel 394,Pixel 395,Pixel 396,Pixel 397,Pixel 398,Pixel 399,Pixel 400,Pixel 401,Pixel 402,Pixel 403,Pixel 404,Pixel 405,Pixel 406,Pixel 407,Pixel 408,Pixel 409,Pixel 410,P

In [15]:
print(df.head())

   label  Pixel 1  Pixel 2  Pixel 3  Pixel 4  Pixel 5  Pixel 6  Pixel 7  \
0      5        0        0        0        0        0        0        0   
1      0        0        0        0        0        0        0        0   
2      4        0        0        0        0        0        0        0   
3      1        0        0        0        0        0        0        0   
4      9        0        0        0        0        0        0        0   

   Pixel 8  Pixel 9  ...  Pixel 775  Pixel 776  Pixel 777  Pixel 778  \
0        0        0  ...          0          0          0          0   
1        0        0  ...          0          0          0          0   
2        0        0  ...          0          0          0          0   
3        0        0  ...          0          0          0          0   
4        0        0  ...          0          0          0          0   

   Pixel 779  Pixel 780  Pixel 781  Pixel 782  Pixel 783  Pixel 784  
0          0          0          0          0 

Let's try to recognize number 5 fromo number 9. Let's try!

In [16]:
numbers = df[df['label'].isin([5, 9])]
numbers.head()

,label,Pixel 1,Pixel 2,Pixel 3,Pixel 4,Pixel 5,Pixel 6,Pixel 7,Pixel 8,Pixel 9,...,Pixel 775,Pixel 776,Pixel 777,Pixel 778,Pixel 779,Pixel 780,Pixel 781,Pixel 782,Pixel 783,Pixel 784
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
len(numbers)

13271

In [17]:
y = numbers['label'].to_dask_array(lengths=True)
X = numbers[numbers.columns.difference(['label'])].to_dask_array(lengths=True)
X = X[0:400]
y = y[0:400]
print(X[2].compute())

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0   31   40 -127    0  -22  -22  -97    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0   68 -106  -17   -2   -3   -3   -3    0
  -41    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0 -100    0  -55   -2   -2   -2  -15 -106
   98 

In [18]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2, train_size=0.8)

In [19]:
def build_deep_neural_network():
    # create model
    
    model = Sequential()
    #model.add(Dense(784, input_shape=(784,), kernel_initializer='normal', activation='relu')) #layer 1 with 60 neurons
    #model.add(Dense(30, kernel_initializer='normal', activation='relu')) #layer 2 with 30 neurons
    #model.add(Dense(1, kernel_initializer='normal', activation='sigmoid')) #layer 3 with 1 neuron (binary classification)
    model.add(Dense(784, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
from sklearn.metrics import accuracy_score
from keras.layers import Dropout
model = build_deep_neural_network()

model.fit(X_train, y_train, verbose=1, epochs=10)
print(model.predict(X_test))
scores = model.evaluate(X_test,y_test, verbose=1)
print("Accuracy: "+str(scores[1]))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/10


InternalError: GPU sync failed